In [ ]:
"""
Script: prefilter_hepa_publication_figures.py

Purpose
-------
Generate publication-ready figures from FIVE repeated dust-loading trials by showing:
  FIG 1) Gross ΔP normalized to a constant reference face velocity (v_ref) vs time
         - bottom x-axis: experiment time (s)
         - top x-axis: equivalent Martian sols
         - right y-axis: Mars-scaled ΔP (optional secondary axis)

  FIG 2) Component ΔP curves (HEPA dp1 and Prefilter dp2) normalized to v_ref vs time
         - bottom x-axis: experiment time (s)
         - top x-axis: equivalent Martian sols
         - right y-axis: Mars-scaled ΔP (optional secondary axis)

  FIG 3) Prefilter ΔP @ v_ref vs cumulative dust loading
         - x-axis: either areal loading (kg/m²) or cumulative dust fed (g)
         - no sols axis; no Mars y-axis

  FIG 4) Face velocity vs time
         - bottom x-axis: experiment time (s)
         - top x-axis: equivalent Martian sols
         - no Mars y-axis

What this script does
---------------------
- Reads 5 CSV "trials" pasted into raw1...raw5 (same time grid required).
- Cleans data (numeric coercion, drop NaNs, sort by time).
- Optionally smooths signals (ΔP and velocity) with a centered rolling mean.
- Reconstructs each ΔP curve to a constant reference velocity using linear Darcy scaling:
      ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))
- Computes pointwise trial-to-trial variability (±1 SD, ddof=1) and optionally smooths SD bands.
- Uses Trial 1 as the main line (consistent with “Trial 1 ± 1 SD across trials” style),
  and overlays ±1 SD bands computed across all 5 trials.
- Exports each figure to PNG (raster) and PDF (vector) in ./fig_exports.

Input CSV format (per raw block)
--------------------------------
Columns required (header row required):
  time, dp1, dp2, velocity

Typical units:
  time in seconds (s)
  dp1, dp2 in Pascals (Pa)
  velocity in meters per second (m/s)

Notes / assumptions
-------------------
- All trials must share the exact same time array (same values in same order).
  If not, resample/align before using this script.
- “Mars-scaled ΔP” secondary axis uses a simple linear scaling for already v_ref-normalized curves:
      ΔP_mars ≈ ΔP_const * (v_mars / v_ref) * slip_factor
  where slip_factor is a user-chosen correction (e.g., 0.85–0.90).
- For dust-loading axis, total fed dust is computed from a constant feed rate and elapsed time.

Environment
-----------
Python 3.10+ recommended; requires:
  numpy, pandas, matplotlib

"""

from __future__ import annotations

from dataclasses import dataclass
from io import StringIO
from pathlib import Path
from typing import Callable

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt


# =============================================================================
# USER SETTINGS
# =============================================================================

@dataclass(frozen=True)
class Settings:
    # Constant-velocity reconstruction reference
    v_ref: float = 0.75  # m/s

    # Time -> Sol scaling (user-provided mapping)
    sol_ref_seconds: float = 2200.0
    sol_ref_sols: float = 68.7

    # Mars-context ΔP scaling (secondary y-axis)
    v_mars_target: float = 0.071  # m/s
    slip_factor: float = 0.875    # 0.85–0.90 typical

    # Smoothing (centered rolling mean); 1 disables
    roll_win: int = 1
    sd_smooth_win: int = 1

    # Robustness
    min_vel: float = 1e-6

    # Dust loading plot inputs
    dust_feed_g_per_s: float = 0.0091      # g/s
    prefilter_area_m2: float | None = 0.00543169  # m^2 (None -> cumulative grams)

    # Output
    outdir: Path = Path("fig_exports")
    dpi: int = 300

    # Figure styling
    fig_w: float = 6.4
    fig_h: float = 4.2
    line_width: float = 1.6
    band_alpha: float = 0.20

    # Typography
    axis_label_size: int = 13
    title_size: int = 14
    tick_label_size: int = 11
    legend_size: int = 10

    # Toggles
    include_mars_yaxis: bool = True


S = Settings()


# =============================================================================
# COLORS (Okabe–Ito, colorblind-safe)
# =============================================================================
C_PREF  = "#E69F00"  # orange
C_HEPA  = "#0072B2"  # blue
C_GROSS = "#009E73"  # green
C_VEL   = "#000000"  # black


# =============================================================================
# LABELS
# =============================================================================
YL_PRESSURE  = "ΔP (Pa)"
YL_VELOCITY  = "Face velocity (m/s)"
BAND_LABEL   = "±1 SD (5 trials)"
XL_TIME      = "Time (s)"
XL_SOLS      = "Equivalent Martian sols"


# =============================================================================
# DATA: paste FIVE CSV trials below
# =============================================================================
raw1 = """
time,dp1,dp2,velocity











"""

raw2 = """
time,dp1,dp2,velocity







"""

raw3 = """
time,dp1,dp2,velocity







"""

raw4 = """
time,dp1,dp2,velocity







"""

raw5 = """
time,dp1,dp2,velocity







"""

RAW_TRIALS = [raw1.strip(), raw2.strip(), raw3.strip(), raw4.strip(), raw5.strip()]


# =============================================================================
# MATPLOTLIB STYLE (publication defaults)
# =============================================================================
def set_publication_style() -> None:
    mpl.rcParams.update({
        # Export
        "savefig.dpi": S.dpi,
        "savefig.bbox": "tight",
        "savefig.pad_inches": 0.02,

        # Fonts (keep default sans; journal templates often override anyway)
        "font.size": S.tick_label_size,
        "axes.titlesize": S.title_size,
        "axes.labelsize": S.axis_label_size,
        "xtick.labelsize": S.tick_label_size,
        "ytick.labelsize": S.tick_label_size,
        "legend.fontsize": S.legend_size,

        # Lines
        "lines.linewidth": S.line_width,
        "axes.linewidth": 1.0,

        # PDF friendliness
        "pdf.fonttype": 42,  # TrueType
        "ps.fonttype": 42,
    })


# =============================================================================
# HELPERS
# =============================================================================
def smooth(arr: np.ndarray, win: int) -> np.ndarray:
    """Centered rolling-mean smoothing. win < 2 disables smoothing."""
    x = np.asarray(arr, dtype=float)
    if win is None or win < 2:
        return x
    return pd.Series(x).rolling(win, center=True, min_periods=1).mean().to_numpy()


def load_trial_csv(raw: str) -> pd.DataFrame:
    """Parse a raw CSV string into a cleaned numeric DataFrame."""
    if not raw.strip():
        raise ValueError("Encountered an empty trial block. Paste CSV data into raw1...raw5.")

    df = pd.read_csv(StringIO(raw), comment="#")
    required = ["time", "dp1", "dp2", "velocity"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}. Required: {required}")

    df = (
        df[required]
        .apply(pd.to_numeric, errors="coerce")
        .dropna()
        .sort_values("time")
        .reset_index(drop=True)
    )
    if df.empty:
        raise ValueError("Trial CSV parsed, but no numeric rows remained after cleaning.")

    return df


def reconstruct_const_velocity_linear_darcy(
    dp: np.ndarray,
    v: np.ndarray,
    v_ref: float,
    min_vel: float,
) -> np.ndarray:
    """
    Linear Darcy reconstruction:
        ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))
    """
    v_use = np.maximum(np.asarray(v, dtype=float), float(min_vel))
    return np.asarray(dp, dtype=float) * (float(v_ref) / v_use)


def sols_per_sec(settings: Settings) -> float:
    return settings.sol_ref_sols / settings.sol_ref_seconds


def add_secondary_sols_axis(ax: mpl.axes.Axes, sols_per_sec_val: float) -> mpl.axes.Axes:
    """Top x-axis: seconds <-> equivalent Martian sols."""
    def sec_to_sols(x): return x * sols_per_sec_val
    def sols_to_sec(x): return x / sols_per_sec_val

    secax = ax.secondary_xaxis("top", functions=(sec_to_sols, sols_to_sec))
    secax.set_xlabel(XL_SOLS)
    return secax


def add_secondary_mars_dp_axis(
    ax: mpl.axes.Axes,
    v_ref: float,
    v_mars: float,
    slip_factor: float,
    label: str,
) -> mpl.axes.Axes:
    """
    Right y-axis for Mars-scaled ΔP.

    Assumes primary y data are already normalized to v_ref (i.e., ΔP_const).
    Uses:
        ΔP_mars ≈ ΔP_const * (v_mars / v_ref) * slip_factor
    """
    scale = (v_mars / max(v_ref, 1e-12)) * slip_factor

    def earth_to_mars(dp_earth): return dp_earth * scale
    def mars_to_earth(dp_mars): return dp_mars / max(scale, 1e-12)

    secay = ax.secondary_yaxis("right", functions=(earth_to_mars, mars_to_earth))
    secay.set_ylabel(label)
    return secay


def enforce_same_time_grid(t_ref: np.ndarray, t_new: np.ndarray) -> None:
    if not np.array_equal(t_ref, t_new):
        raise ValueError(
            "Time arrays differ between trials. All trials must share the same time grid.\n"
            "Fix: resample/align each trial to the same time vector before using this script."
        )


def stack_sd(arr_list: list[np.ndarray], sd_smooth_win: int) -> np.ndarray:
    """Pointwise SD across trials (ddof=1), optionally smoothed."""
    stack = np.column_stack(arr_list)
    sd = stack.std(axis=1, ddof=1)
    return smooth(sd, sd_smooth_win)


def save_figure(fig: mpl.figure.Figure, outdir: Path, stem: str) -> tuple[Path, Path]:
    outdir.mkdir(parents=True, exist_ok=True)
    png = outdir / f"{stem}.png"
    pdf = outdir / f"{stem}.pdf"
    fig.savefig(png)
    fig.savefig(pdf)
    return png, pdf


# =============================================================================
# LOAD + PROCESS TRIALS
# =============================================================================
@dataclass
class TrialProcessed:
    t: np.ndarray
    v_s: np.ndarray
    dp1_const: np.ndarray
    dp2_const: np.ndarray
    gross_const: np.ndarray


def process_trials(raw_trials: list[str], settings: Settings) -> tuple[TrialProcessed, dict[str, np.ndarray]]:
    """
    Returns:
      - trial1: processed series used as the main trace
      - sds: dict of SD arrays (v_sd, dp1_sd, dp2_sd, gross_sd)
    """
    t_ref: np.ndarray | None = None

    v_s_list: list[np.ndarray] = []
    dp1_const_list: list[np.ndarray] = []
    dp2_const_list: list[np.ndarray] = []
    gross_const_list: list[np.ndarray] = []

    trial1_obj: TrialProcessed | None = None

    for i, raw in enumerate(raw_trials, start=1):
        df = load_trial_csv(raw)

        t = df["time"].to_numpy(dtype=float)
        dp1 = df["dp1"].to_numpy(dtype=float)
        dp2 = df["dp2"].to_numpy(dtype=float)
        v   = df["velocity"].to_numpy(dtype=float)

        if t_ref is None:
            t_ref = t
        else:
            enforce_same_time_grid(t_ref, t)

        # Smooth before reconstruction (optional)
        v_s   = smooth(v, settings.roll_win)
        dp1_s = smooth(dp1, settings.roll_win)
        dp2_s = smooth(dp2, settings.roll_win)
        gross_s = smooth(dp1 + dp2, settings.roll_win)

        # Constant-velocity reconstruction
        dp1_const = reconstruct_const_velocity_linear_darcy(dp1_s, v_s, settings.v_ref, settings.min_vel)
        dp2_const = reconstruct_const_velocity_linear_darcy(dp2_s, v_s, settings.v_ref, settings.min_vel)
        gross_const = reconstruct_const_velocity_linear_darcy(gross_s, v_s, settings.v_ref, settings.min_vel)

        v_s_list.append(v_s)
        dp1_const_list.append(dp1_const)
        dp2_const_list.append(dp2_const)
        gross_const_list.append(gross_const)

        if i == 1:
            trial1_obj = TrialProcessed(
                t=t, v_s=v_s, dp1_const=dp1_const, dp2_const=dp2_const, gross_const=gross_const
            )

    assert t_ref is not None and trial1_obj is not None

    sds = {
        "v_sd": stack_sd(v_s_list, settings.sd_smooth_win),
        "dp1_sd": stack_sd(dp1_const_list, settings.sd_smooth_win),
        "dp2_sd": stack_sd(dp2_const_list, settings.sd_smooth_win),
        "gross_sd": stack_sd(gross_const_list, settings.sd_smooth_win),
    }
    return trial1_obj, sds


# =============================================================================
# FIGURE BUILDERS
# =============================================================================
def format_axes(ax: mpl.axes.Axes) -> None:
    ax.grid(True, linestyle="--", alpha=0.35)
    ax.tick_params(direction="out", length=4, width=1)


def plot_with_sd_band(
    ax: mpl.axes.Axes,
    x: np.ndarray,
    y: np.ndarray,
    y_sd: np.ndarray,
    color: str,
    label: str,
    band_label: str | None = None,
) -> None:
    ax.plot(x, y, color=color, label=label)
    ax.fill_between(x, y - y_sd, y + y_sd, color=color, alpha=S.band_alpha, linewidth=0,
                    label=band_label)


def fig1_gross_vs_time(trial1: TrialProcessed, sds: dict[str, np.ndarray], settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w, settings.fig_h))
    plot_with_sd_band(
        ax=ax,
        x=trial1.t,
        y=trial1.gross_const,
        y_sd=sds["gross_sd"],
        color=C_GROSS,
        label=f"Gross ΔP @ v_ref = {settings.v_ref:.2f} m/s (Trial 1)",
        band_label=BAND_LABEL,
    )

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_PRESSURE)
    ax.set_title("Gross ΔP — Constant Face Velocity Reconstruction")
    format_axes(ax)

    ax.legend(frameon=True)

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    if settings.include_mars_yaxis:
        add_secondary_mars_dp_axis(
            ax=ax,
            v_ref=settings.v_ref,
            v_mars=settings.v_mars_target,
            slip_factor=settings.slip_factor,
            label=f"ΔP (Pa) — Mars-scaled @ {settings.v_mars_target:.3f} m/s",
        )

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "fig1_gross_const_time")


def fig2_components_vs_time(trial1: TrialProcessed, sds: dict[str, np.ndarray], settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w + 0.2, settings.fig_h))

    # HEPA
    ax.plot(trial1.t, trial1.dp1_const, color=C_HEPA, label=f"HEPA ΔP @ v_ref (Trial 1)")
    ax.fill_between(trial1.t, trial1.dp1_const - sds["dp1_sd"], trial1.dp1_const + sds["dp1_sd"],
                    color=C_HEPA, alpha=S.band_alpha, linewidth=0)

    # Prefilter
    ax.plot(trial1.t, trial1.dp2_const, color=C_PREF, label=f"Prefilter ΔP @ v_ref (Trial 1)")
    ax.fill_between(trial1.t, trial1.dp2_const - sds["dp2_sd"], trial1.dp2_const + sds["dp2_sd"],
                    color=C_PREF, alpha=S.band_alpha, linewidth=0, label=BAND_LABEL)

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_PRESSURE)
    ax.set_title("Component ΔP vs Time — Constant Face Velocity Reconstruction")
    format_axes(ax)

    ax.legend(frameon=True)

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    if settings.include_mars_yaxis:
        add_secondary_mars_dp_axis(
            ax=ax,
            v_ref=settings.v_ref,
            v_mars=settings.v_mars_target,
            slip_factor=settings.slip_factor,
            label=f"ΔP (Pa) — Mars-scaled @ {settings.v_mars_target:.3f} m/s",
        )

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "fig2_components_time")


def fig3_prefilter_vs_loading(trial1: TrialProcessed, sds: dict[str, np.ndarray], settings: Settings) -> tuple[Path, Path]:
    # Dust loading axis
    t = trial1.t
    duration_s = float(t[-1] - t[0])

    dust_rate_kg_s = max(settings.dust_feed_g_per_s, 0.0) / 1000.0
    cum_mass_kg = dust_rate_kg_s * (t - t[0])  # start at zero

    if settings.prefilter_area_m2 is None:
        x = cum_mass_kg * 1000.0
        x_label = "Cumulative dust fed (g)"
    else:
        area = float(settings.prefilter_area_m2)
        if area <= 0:
            raise ValueError("prefilter_area_m2 must be > 0, or set to None.")
        x = cum_mass_kg / area
        x_label = "Areal dust loading (kg/m²)"

    fig, ax = plt.subplots(figsize=(settings.fig_w, settings.fig_h))
    plot_with_sd_band(
        ax=ax,
        x=x,
        y=trial1.dp2_const,
        y_sd=sds["dp2_sd"],
        color=C_PREF,
        label=f"Prefilter ΔP @ v_ref = {settings.v_ref:.2f} m/s (Trial 1)",
        band_label=BAND_LABEL,
    )

    ax.set_xlabel(x_label)
    ax.set_ylabel(YL_PRESSURE)
    ax.set_title("Prefilter ΔP vs Dust Loading — Constant Face Velocity Reconstruction")
    format_axes(ax)
    ax.legend(frameon=True)

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "fig3_prefilter_dp_vs_loading")


def fig4_velocity_vs_time(trial1: TrialProcessed, sds: dict[str, np.ndarray], settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w, settings.fig_h))
    plot_with_sd_band(
        ax=ax,
        x=trial1.t,
        y=trial1.v_s,
        y_sd=sds["v_sd"],
        color=C_VEL,
        label="Face velocity (Trial 1)",
        band_label=BAND_LABEL,
    )

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_VELOCITY)
    ax.set_title("Media Face Velocity vs Time")
    format_axes(ax)
    ax.legend(frameon=True)

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "fig4_velocity_vs_time")


# =============================================================================
# NUMERICAL SUMMARY (for manuscript text)
# =============================================================================
def print_results_summary(trial1: TrialProcessed, settings: Settings) -> None:
    t = trial1.t
    dt = float(t[-1] - t[0])
    sols = dt * sols_per_sec(settings)
    n_trials = len(RAW_TRIALS)

    # Prefilter
    dp2_init = float(trial1.dp2_const[0])
    dp2_final = float(trial1.dp2_const[-1])
    dp2_rise = dp2_final - dp2_init
    dp2_pct = 100.0 * dp2_rise / dp2_init if dp2_init != 0 else np.nan
    dp2_rate = dp2_rise / dt if dt > 0 else np.nan

    # Gross
    gross_init = float(trial1.gross_const[0])
    gross_final = float(trial1.gross_const[-1])
    gross_rise = gross_final - gross_init
    gross_rate = gross_rise / dt if dt > 0 else np.nan

    # Velocity
    v_init = float(trial1.v_s[0])
    v_final = float(trial1.v_s[-1])

    # Time to 50% of total prefilter ΔP rise
    if np.isfinite(dp2_rise) and dp2_rise != 0:
        half_dp = dp2_init + 0.5 * dp2_rise
        idx_half = int(np.argmin(np.abs(trial1.dp2_const - half_dp)))
        t_half = float(t[idx_half] - t[0])  # seconds since start
    else:
        t_half = np.nan

    # Dust
    dust_total_g = settings.dust_feed_g_per_s * dt
    dust_total_kg = dust_total_g / 1000.0

    if settings.prefilter_area_m2 is not None and settings.prefilter_area_m2 > 0:
        areal_loading = dust_total_kg / settings.prefilter_area_m2
        norm_rate = dp2_rise / areal_loading if areal_loading != 0 else np.nan
        norm_str = f"{norm_rate:.2f} Pa / (kg/m²)"
    else:
        areal_loading = None
        norm_rate = dp2_rise / dust_total_g if dust_total_g != 0 else np.nan
        norm_str = f"{norm_rate:.2f} Pa/g"

    print("\n" + "=" * 60)
    print("RESULTS SUMMARY (Trial 1 values; for manuscript text)")
    print("=" * 60)
    print(f"Test duration: {dt:.1f} s  (~{sols:.2f} equivalent Martian sols)")
    print(f"Number of trials: n = {n_trials}")
    print(f"Reconstruction: linear Darcy ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))")
    print(f"v_ref = {settings.v_ref:.3f} m/s")
    if settings.include_mars_yaxis:
        print(f"Mars scaling: v_mars = {settings.v_mars_target:.3f} m/s, slip_factor = {settings.slip_factor:.3f}")

    print("\nPrefilter ΔP (constant face velocity):")
    print(f"  Initial ΔP: {dp2_init:.2f} Pa")
    print(f"  Final ΔP:   {dp2_final:.2f} Pa")
    print(f"  Rise:       {dp2_rise:.2f} Pa  ({dp2_pct:.1f} %)")
    print(f"  Mean loading rate: {dp2_rate:.3f} Pa/s")
    print(f"  Time to 50% ΔP rise: {t_half:.1f} s")

    print("\nFace velocity response:")
    print(f"  Initial velocity: {v_init:.3f} m/s")
    print(f"  Final velocity:   {v_final:.3f} m/s")

    print("\nDust loading:")
    print(f"  Total dust fed: {dust_total_g:.2f} g")
    if areal_loading is not None:
        print(f"  Areal loading: {areal_loading:.3f} kg/m²")
    print(f"  Normalized clogging: {norm_str}")

    print("\nGross system ΔP (constant face velocity):")
    print(f"  Initial ΔP: {gross_init:.2f} Pa")
    print(f"  Final ΔP:   {gross_final:.2f} Pa")
    print(f"  Rise:       {gross_rise:.2f} Pa")
    print(f"  Mean loading rate: {gross_rate:.3f} Pa/s")
    print("=" * 60 + "\n")


# =============================================================================
# MAIN
# =============================================================================
def main() -> None:
    set_publication_style()

    trial1, sds = process_trials(RAW_TRIALS, S)

    p1 = fig1_gross_vs_time(trial1, sds, S)
    p2 = fig2_components_vs_time(trial1, sds, S)
    p3 = fig3_prefilter_vs_loading(trial1, sds, S)
    p4 = fig4_velocity_vs_time(trial1, sds, S)

    print_results_summary(trial1, S)

    print("Saved figures:")
    for (png, pdf) in [p1, p2, p3, p4]:
        print(f"  {png} | {pdf}")


if __name__ == "__main__":
    main()


In [ ]:
"""
Script: baseline_hepa_only_publication_figures.py

Baseline (HEPA-only) — SEPARATE FIGURES (publication-ready)

Outputs (PNG + PDF for each)
----------------------------
FIG A) HEPA ΔP vs Time — Constant Face Velocity (Darcy reconstruction)
       - main curve: Trial 1
       - band: ±1 SD (5 trials)
       - top x-axis: Equivalent Martian sols
       - right y-axis: Mars-scaled ΔP (constant multiplier mapping; optional toggle)

FIG B) HEPA ΔP vs Time — Variable Velocity (measured raw ΔP)
       - main curve: Trial 1
       - band: ±1 SD (5 trials)
       - top x-axis: Equivalent Martian sols
       - (Mars y-axis OFF by default; can be enabled)

FIG C) Face Velocity vs Time
       - main curve: Trial 1
       - band: ±1 SD (5 trials)
       - top x-axis: Equivalent Martian sols

Assumptions / Input
-------------------
- Paste 5 trials with columns: time, dp1, dp2, velocity (dp2 ignored here).
- All trials share the exact same time array (same values in same order).
- Smoothing is a centered rolling mean; set window=1 to disable.

Methods
-------
- Constant-velocity reconstruction uses linear Darcy scaling:
      ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))

- Mars-scaled ΔP secondary axis (for v_ref-normalized curves):
      ΔP_mars ≈ ΔP_const * (v_mars / v_ref) * slip_factor

Environment
-----------
Python 3.10+ recommended; requires:
  numpy, pandas, matplotlib
"""

from __future__ import annotations

from dataclasses import dataclass
from io import StringIO
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt


# =============================================================================
# USER SETTINGS
# =============================================================================

@dataclass(frozen=True)
class Settings:
    # Reconstruction reference
    v_ref: float = 0.75  # m/s

    # Time -> Sol scaling: 2200 s corresponds to 68.7 sols
    sol_ref_seconds: float = 2200.0
    sol_ref_sols: float = 68.7

    # Mars-context ΔP scaling (secondary y-axis)
    v_mars_target: float = 0.071
    slip_factor: float = 0.875

    # Smoothing (centered rolling mean); 1 disables
    roll_win: int = 9
    sd_smooth_win: int = 9

    # Robustness
    min_vel: float = 1e-6

    # Output
    outdir: Path = Path("fig_exports_baseline")
    dpi: int = 300

    # Typography
    axis_label_size: int = 13
    title_size: int = 14
    tick_label_size: int = 11
    legend_size: int = 10

    # Styling
    fig_w: float = 6.6
    fig_h: float = 4.4
    line_width: float = 1.8
    band_alpha: float = 0.20

    # Toggles
    mars_axis_on_figA: bool = True
    mars_axis_on_figB: bool = False


S = Settings()


# =============================================================================
# COLORS (Okabe–Ito, colorblind-safe)
# =============================================================================
C_HEPA = "#0072B2"  # blue
C_VEL = "#000000"   # black


# =============================================================================
# LABELS
# =============================================================================
YL_PRESSURE = "ΔP (Pa)"
YL_VELOCITY = "Face velocity (m/s)"
BAND_LABEL = "±1 SD (5 trials)"
XL_TIME = "Time (s)"
XL_SOLS = "Equivalent Martian sols"


# =============================================================================
# DATA: paste FIVE CSV trials
# =============================================================================
raw1 = """
time,dp1,dp2,velocity







"""

raw2 = """
time,dp1,dp2,velocity







"""

raw3 = """
time,dp1,dp2,velocity







"""

raw4 = """
time,dp1,dp2,velocity







"""

raw5 = """
time,dp1,dp2,velocity







"""

RAW_TRIALS = [raw1.strip(), raw2.strip(), raw3.strip(), raw4.strip(), raw5.strip()]


# =============================================================================
# MATPLOTLIB STYLE (publication defaults)
# =============================================================================
def set_publication_style() -> None:
    mpl.rcParams.update({
        "savefig.dpi": S.dpi,
        "savefig.bbox": "tight",
        "savefig.pad_inches": 0.02,

        "font.size": S.tick_label_size,
        "axes.titlesize": S.title_size,
        "axes.labelsize": S.axis_label_size,
        "xtick.labelsize": S.tick_label_size,
        "ytick.labelsize": S.tick_label_size,
        "legend.fontsize": S.legend_size,

        "lines.linewidth": S.line_width,
        "axes.linewidth": 1.0,

        # PDF friendliness
        "pdf.fonttype": 42,
        "ps.fonttype": 42,
    })


# =============================================================================
# HELPERS
# =============================================================================
def sols_per_sec(settings: Settings) -> float:
    return settings.sol_ref_sols / settings.sol_ref_seconds


def smooth(arr: np.ndarray, win: int) -> np.ndarray:
    """Centered rolling-mean smoothing. win < 2 disables smoothing."""
    x = np.asarray(arr, dtype=float)
    if win is None or win < 2:
        return x
    # keep it odd-ish for symmetry if user accidentally provides even
    if win % 2 == 0:
        win += 1
    return pd.Series(x).rolling(win, center=True, min_periods=1).mean().to_numpy()


def load_trial_csv(raw: str) -> pd.DataFrame:
    """Parse a raw CSV string into a cleaned numeric DataFrame."""
    if not raw.strip():
        raise ValueError("Encountered an empty trial block. Paste CSV data into raw1...raw5.")

    df = pd.read_csv(StringIO(raw), comment="#")
    required = ["time", "dp1", "dp2", "velocity"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(f"Missing required columns: {missing}. Required: {required}")

    df = (
        df[required]
        .apply(pd.to_numeric, errors="coerce")
        .dropna()
        .sort_values("time")
        .reset_index(drop=True)
    )
    if df.empty:
        raise ValueError("Trial CSV parsed, but no numeric rows remained after cleaning.")

    return df


def enforce_same_time_grid(t_ref: np.ndarray, t_new: np.ndarray) -> None:
    if not np.array_equal(t_ref, t_new):
        raise ValueError(
            "Time arrays differ between trials. All trials must share the same time grid.\n"
            "Fix: resample/align each trial to the same time vector before using this script."
        )


def reconstruct_const_velocity_linear_darcy(
    dp: np.ndarray,
    v: np.ndarray,
    v_ref: float,
    min_vel: float,
) -> np.ndarray:
    """Linear Darcy reconstruction: ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))."""
    v_use = np.maximum(np.asarray(v, dtype=float), float(min_vel))
    return np.asarray(dp, dtype=float) * (float(v_ref) / v_use)


def add_secondary_sols_axis(ax: mpl.axes.Axes, sols_per_sec_val: float) -> mpl.axes.Axes:
    """Top x-axis: seconds <-> equivalent Martian sols."""
    def sec_to_sols(x): return x * sols_per_sec_val
    def sols_to_sec(x): return x / sols_per_sec_val

    secax = ax.secondary_xaxis("top", functions=(sec_to_sols, sols_to_sec))
    secax.set_xlabel(XL_SOLS)
    return secax


def add_secondary_mars_dp_axis(
    ax: mpl.axes.Axes,
    v_ref: float,
    v_mars: float,
    slip_factor: float,
    label: str,
) -> mpl.axes.Axes:
    """
    Right y-axis for Mars-scaled ΔP.

    Assumes primary y data are already normalized to v_ref (i.e., ΔP_const).
    Uses:
        ΔP_mars ≈ ΔP_const * (v_mars / v_ref) * slip_factor
    """
    scale = (v_mars / max(v_ref, 1e-12)) * slip_factor

    def earth_to_mars(dp_earth): return dp_earth * scale
    def mars_to_earth(dp_mars): return dp_mars / max(scale, 1e-12)

    secay = ax.secondary_yaxis("right", functions=(earth_to_mars, mars_to_earth))
    secay.set_ylabel(label)
    return secay


def stack_sd(arr_list: list[np.ndarray], sd_smooth_win: int) -> np.ndarray:
    """Pointwise SD across trials (ddof=1), optionally smoothed."""
    stack = np.column_stack(arr_list)
    sd = stack.std(axis=1, ddof=1)
    return smooth(sd, sd_smooth_win)


def format_axes(ax: mpl.axes.Axes) -> None:
    ax.grid(True, linestyle="--", alpha=0.35)
    ax.tick_params(direction="out", length=4, width=1)


def save_figure(fig: mpl.figure.Figure, outdir: Path, stem: str) -> tuple[Path, Path]:
    outdir.mkdir(parents=True, exist_ok=True)
    png = outdir / f"{stem}.png"
    pdf = outdir / f"{stem}.pdf"
    fig.savefig(png)
    fig.savefig(pdf)
    return png, pdf


# =============================================================================
# LOAD + PROCESS TRIALS
# =============================================================================
@dataclass
class BaselineProcessed:
    t: np.ndarray
    dp1_raw: np.ndarray
    dp1_const: np.ndarray
    v: np.ndarray
    dp1_raw_sd: np.ndarray
    dp1_const_sd: np.ndarray
    v_sd: np.ndarray


def process_baseline(raw_trials: list[str], settings: Settings) -> BaselineProcessed:
    t_ref: np.ndarray | None = None

    dp1_raw_list: list[np.ndarray] = []
    dp1_const_list: list[np.ndarray] = []
    v_list: list[np.ndarray] = []

    trial1_t: np.ndarray | None = None
    trial1_dp1_raw: np.ndarray | None = None
    trial1_dp1_const: np.ndarray | None = None
    trial1_v: np.ndarray | None = None

    for i, raw in enumerate(raw_trials, start=1):
        df = load_trial_csv(raw)

        t = df["time"].to_numpy(dtype=float)
        dp1 = df["dp1"].to_numpy(dtype=float)
        v = df["velocity"].to_numpy(dtype=float)

        if t_ref is None:
            t_ref = t
        else:
            enforce_same_time_grid(t_ref, t)

        # Smooth measured signals
        v_s = smooth(v, settings.roll_win)
        dp1_s = smooth(dp1, settings.roll_win)

        # Constant-velocity reconstruction
        dp1_const = reconstruct_const_velocity_linear_darcy(dp1_s, v_s, settings.v_ref, settings.min_vel)

        dp1_raw_list.append(dp1_s)
        dp1_const_list.append(dp1_const)
        v_list.append(v_s)

        if i == 1:
            trial1_t = t
            trial1_dp1_raw = dp1_s
            trial1_dp1_const = dp1_const
            trial1_v = v_s

    assert t_ref is not None and trial1_t is not None
    assert trial1_dp1_raw is not None and trial1_dp1_const is not None and trial1_v is not None

    dp1_raw_sd = stack_sd(dp1_raw_list, settings.sd_smooth_win)
    dp1_const_sd = stack_sd(dp1_const_list, settings.sd_smooth_win)
    v_sd = stack_sd(v_list, settings.sd_smooth_win)

    return BaselineProcessed(
        t=trial1_t,
        dp1_raw=trial1_dp1_raw,
        dp1_const=trial1_dp1_const,
        v=trial1_v,
        dp1_raw_sd=dp1_raw_sd,
        dp1_const_sd=dp1_const_sd,
        v_sd=v_sd,
    )


# =============================================================================
# FIGURES
# =============================================================================
def figA_hepa_const_time(proc: BaselineProcessed, settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w, settings.fig_h))

    ax.plot(proc.t, proc.dp1_const, color=C_HEPA, label="HEPA ΔP (Trial 1)")
    ax.fill_between(
        proc.t,
        proc.dp1_const - proc.dp1_const_sd,
        proc.dp1_const + proc.dp1_const_sd,
        color=C_HEPA,
        alpha=settings.band_alpha,
        linewidth=0,
        label=BAND_LABEL,
    )

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_PRESSURE)
    ax.set_title("HEPA ΔP vs Time — Constant Face Velocity (Darcy Reconstruction)")
    format_axes(ax)
    ax.legend(frameon=True, loc="upper left")

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    if settings.mars_axis_on_figA:
        add_secondary_mars_dp_axis(
            ax=ax,
            v_ref=settings.v_ref,
            v_mars=settings.v_mars_target,
            slip_factor=settings.slip_factor,
            label=f"ΔP (Pa) — Mars-scaled @ {settings.v_mars_target:.3f} m/s",
        )

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "baseline_figA_hepa_dp_const_time")


def figB_hepa_raw_time(proc: BaselineProcessed, settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w, settings.fig_h))

    ax.plot(proc.t, proc.dp1_raw, color=C_HEPA, label="HEPA ΔP (Trial 1)")
    ax.fill_between(
        proc.t,
        proc.dp1_raw - proc.dp1_raw_sd,
        proc.dp1_raw + proc.dp1_raw_sd,
        color=C_HEPA,
        alpha=settings.band_alpha,
        linewidth=0,
        label=BAND_LABEL,
    )

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_PRESSURE)
    ax.set_title("HEPA ΔP vs Time — Variable Velocity (Measured)")
    format_axes(ax)
    ax.legend(frameon=True, loc="upper left")

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    # Off by default (matches your spec), but you can enable via Settings.mars_axis_on_figB
    if settings.mars_axis_on_figB:
        add_secondary_mars_dp_axis(
            ax=ax,
            v_ref=settings.v_ref,
            v_mars=settings.v_mars_target,
            slip_factor=settings.slip_factor,
            label=f"ΔP (Pa) — Mars-scaled @ {settings.v_mars_target:.3f} m/s",
        )

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "baseline_figB_hepa_dp_variable_time")


def figC_velocity_time(proc: BaselineProcessed, settings: Settings) -> tuple[Path, Path]:
    fig, ax = plt.subplots(figsize=(settings.fig_w + 0.2, settings.fig_h))

    ax.plot(proc.t, proc.v, color=C_VEL, label="Face velocity (Trial 1)")
    ax.fill_between(
        proc.t,
        proc.v - proc.v_sd,
        proc.v + proc.v_sd,
        color=C_VEL,
        alpha=settings.band_alpha * 0.9,
        linewidth=0,
        label=BAND_LABEL,
    )

    ax.set_xlabel(XL_TIME)
    ax.set_ylabel(YL_VELOCITY)
    ax.set_title("Face Velocity vs Time")
    format_axes(ax)
    ax.legend(frameon=True, loc="upper right")

    add_secondary_sols_axis(ax, sols_per_sec(settings))

    fig.tight_layout()
    return save_figure(fig, settings.outdir, "baseline_figC_velocity_time")


# =============================================================================
# NUMERICAL SUMMARY (for manuscript text)
# =============================================================================
def print_results_summary(proc: BaselineProcessed, settings: Settings) -> None:
    t = proc.t
    duration_s = float(t[-1] - t[0])
    duration_sols = duration_s * sols_per_sec(settings)
    n_trials = len(RAW_TRIALS)

    # HEPA ΔP (variable velocity / measured)
    dp1_raw_init = float(proc.dp1_raw[0])
    dp1_raw_final = float(proc.dp1_raw[-1])
    dp1_raw_rise = dp1_raw_final - dp1_raw_init
    dp1_raw_pct = 100.0 * dp1_raw_rise / dp1_raw_init if dp1_raw_init != 0 else np.nan
    dp1_raw_rate = dp1_raw_rise / duration_s if duration_s > 0 else np.nan

    half_raw = dp1_raw_init + 0.5 * dp1_raw_rise
    idx_half_raw = int(np.argmin(np.abs(proc.dp1_raw - half_raw)))
    t_half_raw = float(t[idx_half_raw] - t[0])

    # HEPA ΔP (constant face velocity reconstruction)
    dp1_const_init = float(proc.dp1_const[0])
    dp1_const_final = float(proc.dp1_const[-1])
    dp1_const_rise = dp1_const_final - dp1_const_init
    dp1_const_pct = 100.0 * dp1_const_rise / dp1_const_init if dp1_const_init != 0 else np.nan
    dp1_const_rate = dp1_const_rise / duration_s if duration_s > 0 else np.nan

    half_const = dp1_const_init + 0.5 * dp1_const_rise
    idx_half_const = int(np.argmin(np.abs(proc.dp1_const - half_const)))
    t_half_const = float(t[idx_half_const] - t[0])

    # Velocity response
    v_init = float(proc.v[0])
    v_final = float(proc.v[-1])

    # Reconstruction amplification at end
    amp_end = (dp1_const_final / dp1_raw_final) if dp1_raw_final != 0 else np.nan

    print("\n" + "=" * 60)
    print("RESULTS SUMMARY (Trial 1 values; for manuscript text) — BASELINE (HEPA-only)")
    print("=" * 60)
    print(f"Test duration: {duration_s:.1f} s  (~{duration_sols:.2f} equivalent Martian sols)")
    print(f"Number of trials: n = {n_trials}")
    print(f"Reconstruction: ΔP_const(t) = ΔP_meas(t) * (v_ref / v(t))")
    print(f"v_ref = {settings.v_ref:.3f} m/s")

    print("\nHEPA ΔP (variable velocity; measured):")
    print(f"  Initial ΔP: {dp1_raw_init:.2f} Pa")
    print(f"  Final ΔP:   {dp1_raw_final:.2f} Pa")
    print(f"  Rise:       {dp1_raw_rise:.2f} Pa  ({dp1_raw_pct:.1f} %)")
    print(f"  Mean loading rate: {dp1_raw_rate:.3f} Pa/s")
    print(f"  Time to 50% ΔP rise: {t_half_raw:.1f} s")

    print("\nHEPA ΔP (constant face velocity; Darcy reconstruction):")
    print(f"  Initial ΔP: {dp1_const_init:.2f} Pa")
    print(f"  Final ΔP:   {dp1_const_final:.2f} Pa")
    print(f"  Rise:       {dp1_const_rise:.2f} Pa  ({dp1_const_pct:.1f} %)")
    print(f"  Mean loading rate: {dp1_const_rate:.3f} Pa/s")
    print(f"  Time to 50% ΔP rise: {t_half_const:.1f} s")
    print(f"  Const/Measured ΔP ratio at end: {amp_end:.2f}×")

    print("\nFace velocity response:")
    print(f"  Initial velocity: {v_init:.3f} m/s")
    print(f"  Final velocity:   {v_final:.3f} m/s")
    print("=" * 60 + "\n")


# =============================================================================
# MAIN
# =============================================================================
def main() -> None:
    set_publication_style()

    S.outdir.mkdir(parents=True, exist_ok=True)

    proc = process_baseline(RAW_TRIALS, S)

    (a_png, a_pdf) = figA_hepa_const_time(proc, S)
    (b_png, b_pdf) = figB_hepa_raw_time(proc, S)
    (c_png, c_pdf) = figC_velocity_time(proc, S)

    print_results_summary(proc, S)

    print("Saved figures:")
    print(f"  {a_png} | {a_pdf}")
    print(f"  {b_png} | {b_pdf}")
    print(f"  {c_png} | {c_pdf}")


if __name__ == "__main__":
    main()
